In [2]:
from pystruct.models import LatentNodeCRF
from pystruct.models import GraphCRF
from pystruct.learners import FrankWolfeSSVM
from pystruct.learners import OneSlackSSVM
from pystruct.learners import NSlackSSVM
from pystruct.learners import LatentSSVM
from pystruct.learners import SubgradientLatentSSVM
from pystruct.datasets import load_letters
from pystruct.utils import make_grid_edges
import numpy as np
import itertools
import pandas as pd
from sklearn.model_selection import train_test_split

## Convert seqs file into a character array

In [3]:
msa = np.array([np.array(list(x), dtype=object) for x in open('adk_lid_uppercase.seqs', 'r').read().split('\n') if x], dtype=object)
default_len = len(msa[0])

### Array format demo

In [4]:
pd.DataFrame(msa)

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,R,W,Y,H,P,K,-,S,G,R,...,V,Q,-,-,-,H,S,E,-,D
1,R,C,T,H,P,A,-,S,G,R,...,V,W,-,-,-,R,-,D,-,D
2,R,R,A,H,L,P,-,S,G,R,...,V,I,-,-,-,R,-,E,-,D
3,R,R,A,H,L,P,-,S,G,R,...,V,V,-,-,-,R,-,D,-,D
4,R,R,A,H,L,P,-,S,G,R,...,V,V,-,-,-,R,-,D,-,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,R,Y,S,C,K,N,-,C,G,K,...,D,Y,-,-,-,R,-,K,-,D
295,R,Y,S,C,K,N,-,C,G,K,...,D,Y,-,-,-,R,-,K,-,D
296,R,Y,S,C,K,N,-,C,G,K,...,D,Y,-,-,-,R,-,K,-,D
297,R,L,V,C,K,N,-,C,G,M,...,Y,R,-,-,-,R,-,P,-,D


## Load controls

In [5]:
df = pd.read_csv('control_seqs.csv')

## Convert controls into a character array

In [6]:
controls = np.array([list(i[0:default_len]) for i in df.seq if len(i) >= default_len], dtype=object)

## Combine msa and controls

In [7]:
combination = np.concatenate((controls, msa))

In [8]:
#create library of all amino acids
library = {'-': 0, 'C': 1, 'D': 2, 'S': 3,
 'Q': 4, 'K': 5, 'I': 6, 'P': 7,
 'T': 8, 'F': 9, 'N': 10, 'G': 11,
 'H': 12, 'L': 13, 'R': 14, 'W': 15,
 'A': 16, 'V': 17, 'E': 18, 'Y': 19,
 'M': 20, 'X': 21}

len(library)

22

### Set all latent edges so that the final output is a 1D array

In [20]:
l = set(range(default_len+1))
latent_edges = np.array([list(i) for i in itertools.combinations(l, 2)])
latent_edges.shape

(1431, 2)

## One hot encode msa values 

In [21]:
X = []
for i in combination:
    node = [[0 for _ in range(len(library))]]
    for k in i:
        z = np.zeros(len(library), dtype=int)
        z[library[k]] = 1
        node.append(z)
    X.append(np.array(node))
X=np.array(X)
X.shape

(672, 54, 22)

In [22]:
X_ = list(zip(X, [latent_edges for _ in X], [default_len for _ in X]))

# X, latent_edges per sample, Number of Hidden nodes
# number of hidden nodes reduced the msa x length down to one dimension

In [23]:
y_controls = np.zeros(shape = (len(controls), 1), dtype=int)
y_msa = np.ones(shape = (len(msa), 1), dtype=int)
y = np.concatenate((y_controls, y_msa))

In [24]:
print(y_controls.shape)
print(y_msa.shape)


(373, 1)
(299, 1)


In [25]:
# split the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=.2)

In [26]:
print(len(X_train)) # out of the total controls and targets, this was used to train the model
print(len(X_test))


537
135


In [27]:
len(library) # library of amino acids == features

22

In [28]:
model = LatentNodeCRF(n_labels=2, n_features=len(library), latent_node_features=True)

In [29]:
ssvm = OneSlackSSVM(model, max_iter=200, C=100, n_jobs=-1, show_loss_every=10, inference_cache=50)

latentssvm = LatentSSVM(base_ssvm=ssvm)

In [30]:
import time
tic = time.time()
latentssvm.fit(X_train, y_train)
tic2 = time.time() 
runtime = tic2 - tic
print(f'Runtime for training: {runtime}s')

C:\ProgramData\Anaconda3\lib\site-packages\pystruct-0.3.2-py3.8-win-amd64.egg\pystruct\learners\one_slack_ssvm.py:307: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.all(y_1 == y_2)


Runtime for training: 401.0895006656647s


In [31]:
latentssvm.score(X_test, y_test)

0.9925925925925926

In [32]:
pd.options.display.max_rows = 200

In [33]:
pd.DataFrame([latentssvm.predict(X_test), y_test]).T.head()


,0,1
0,[1],[1]
1,[0],[0]
2,[0],[0]
3,[1],[1]
4,[0],[0]


# 99.26% accuracy!

In [34]:
latentssvm.w
#?????? Not sure what this is

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -7.34422902e-03,  2.75922847e-01,  4.19481769e-01, -1.13986036e-01,
       -2.03638625e-01, -7.05456726e-02,  4.35586202e-02,  4.73904235e-01,
       -5.39124713e-02, -

In [35]:
pairwise = model._get_pairwise_potentials(X_[108], latentssvm.w)
pd.DataFrame(pairwise)

,0,1,2,3
0,0.000000,0.000000,-0.009391,0.161777
1,0.000000,0.000000,0.009501,-0.161886
2,-0.009391,0.009501,0.012079,-0.018458
3,0.161777,-0.161886,-0.018458,0.006379


In [36]:
combination[33]

array(['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', 'M', 'A', 'Q', 'I', 'K', 'I', 'T', 'L', 'T', 'K', 'S', 'P',
       'I', 'G', 'R', 'I', 'P', 'S', 'Q', 'R', 'K', 'T', 'V', 'V', 'A',
       'L', 'G', 'L', 'G', 'K', 'L', 'N', 'G', 'S', 'V', 'I', 'K', 'E',
       'D'], dtype=object)

In [37]:
pairwise = model._get_unary_potentials(X_[33], latentssvm.w)
pd.DataFrame(pairwise)

,0,1,2,3
0,0.0,0.0,-100.000000,-100.000000
1,-100.0,-100.0,-0.007344,0.007344
2,-100.0,-100.0,-0.007344,0.007344
3,-100.0,-100.0,-0.007344,0.007344
4,-100.0,-100.0,-0.007344,0.007344
5,-100.0,-100.0,-0.007344,0.007344
6,-100.0,-100.0,-0.007344,0.007344
7,-100.0,-100.0,-0.007344,0.007344
8,-100.0,-100.0,-0.007344,0.007344
9,-100.0,-100.0,-0.007344,0.007344
